In [1]:
import os
os.chdir("D:\Projects\LEGO\Rebrickable")

In [2]:
import pandas as pd
inventories = pd.read_csv("inventories.csv")
inv_p = pd.read_csv("inventory_parts.csv")
inv_s = pd.read_csv("inventory_sets.csv")

In [3]:
inventories.head()

,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1
2,4,1,6942-1
3,15,1,5158-1
4,16,1,903-1


In [4]:
inv_p.head()

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,mcsport6,25,1,f
3,1,paddle,0,1,f
4,3,11816pr0005,78,1,f


In [5]:
inv_s.head()

,inventory_id,set_num,quantity
0,35,75911-1,1
1,35,75912-1,1
2,39,75048-1,1
3,39,75053-1,1
4,50,4515-1,1


### merge sets

In [6]:
inv = inv_p.merge(inventories, left_on="inventory_id",right_on = "id", how="left")
inv.head()

,inventory_id,part_num,color_id,quantity,is_spare,id,version,set_num
0,1,48379c01,72,1,f,1,1,7922-1
1,1,48395,7,1,f,1,1,7922-1
2,1,mcsport6,25,1,f,1,1,7922-1
3,1,paddle,0,1,f,1,1,7922-1
4,3,11816pr0005,78,1,f,3,1,3931-1


In [7]:
inv = inv.drop("id",axis=1)

In [8]:
inv[inv.version==5].set_num.unique()

array(['421-2', '700.1.1-1', '700.1.2-1'], dtype=object)

In [9]:
# return drop index list
def update_version(v):
    uni = inv[inv.version==v].set_num.unique()
    ind = inv[(inv.version != v) & (inv.set_num.isin(uni))].index
    return ind

In [10]:
v5 = update_version(5)
inv = inv.drop(v5)

In [11]:
v5

Int64Index([394072, 402767, 484000, 484001, 484002, 484003, 484004, 575967,
            575968, 575969, 575970, 575971, 575972, 575973, 575974, 575975,
            575976, 575977, 575978, 575979, 575980, 575981, 649854, 649855,
            649856, 649858, 649859, 649860],
           dtype='int64')

In [12]:
v4 = update_version(4)
inv = inv.drop(v4)
v3 = update_version(3)
inv = inv.drop(v3)
v2 = update_version(2)
inv = inv.drop(v2)

In [14]:
df = inv.groupby(["part_num","set_num"], as_index=False).agg({"quantity":"sum"})

In [15]:
df.head()

,part_num,set_num,quantity
0,004591,1650-1,1
1,004602,1650-1,1
2,0687b1,970687-1,1
3,0901,080-1,1
4,0901,080-2,1


In [17]:
df = df.pivot(index='part_num',columns='set_num',values='quantity')
df.head()

set_num,001-1,0011-2,0012-1,0013-1,0014-1,0015-1,0016-1,002-1,003-1,005-1,...,lwp05-1,lwp06-1,mln09-1,sw117promo-1,sw218promo-1,tech007promo-1,test-1,tf05-1,trucapam-1,vwkit-1
part_num,,,,,,,,,,,,,,,,,,,,,
004591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
004602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0687b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# calculate similarity
# porsche 911 42056-1, detective's offic: 10246-1, mickey mouse: 41624-1, minnie mouse: 41625-1, the beatles yellow submarine: 21306-1,
# downtown dinner: 10260-1, thanos: 41605-1, iron man: 41604-1, steamboat willie: 21317-1
owned_sets = ["42056-1","10246-1","41624-1","41625-1","21306-1","10260-1","41605-1","41604-1","21317-1"]
set_o = df.loc[:,owned_sets]

In [32]:
set_o["my_sets"] = set_o.sum(axis=1,skipna=True)

In [34]:
potential_sets = df.drop(owned_sets,axis=1)

In [35]:
sets = potential_sets.columns

In [39]:
potential_sets = potential_sets.fillna(0)

In [51]:
complete = []
my_sets = set_o["my_sets"]

for i in sets:
    compare_set = potential_sets[i]
    parts_num = compare_set.sum()
    gap = compare_set - my_sets
    parts_got = parts_num - gap[gap>0].sum()
    percent = round(100*parts_got/parts_num,2)
    complete.append(percent)

In [59]:
d = {'set_num': sets, 'complete_percent': complete}
df2 = pd.DataFrame(d).sort_values(by='complete_percent',ascending=False)

In [54]:
set_df = pd.read_csv("sets.csv")
set_df.head()

,set_num,name,year,theme_id,num_parts
0,001-1,Gears,1965,1,43
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,2
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [63]:
df3 = df2.merge(set_df,on="set_num",how="left")
df3 = df3.dropna()

In [68]:
stop_words = ["Calendar","Education","Bricks","Brick"]

In [71]:
com_df = df3[df3.complete_percent == 100]

def check(item):
    result = "t"
    if ("Brick" in item) or ("Calendar" in item) or ("Education" in item) or ("Plates" in item) or ("LEGOLAND" in item):
        result = "f"
    return result

com_df["true_set"] = com_df.name.apply(check)
com_df2 = com_df[com_df.true_set == "t"]
com_df2

D:\conda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,set_num,complete_percent,name,year,theme_id,num_parts,true_set
7,e1a1401-1,100.0,MUJI Basic Set,2011,301,64,t
10,10106-1,100.0,Snowflake,2006,227,107,t
17,comcon026-1,100.0,Kraang - New York Comic-Con 2012 Exclusive,2012,570,1,t
19,LLCAPUM2-1,100.0,Build N Buy Pumpkin (Sad) (Legoland California),2002,427,29,t
22,4659597-1,100.0,Friends - Bracelets,2012,501,4,t
26,40219-1,100.0,Monthly Mini Model Build December 2016 - Chris...,2016,409,55,t
31,970-2,100.0,47 1/3 elements (plates),1969,371,141,t
47,fruit1-1,100.0,Apple - Hong Kong Lego Show Promotional,2007,33,8,t
50,WHITEHOUSE-1,100.0,Micro White House,2015,598,59,t
51,421-2,100.0,2 eights 2 sixes 2 fours 8 twos 6 ones (The Bu...,1966,371,20,t


In [73]:
com_df2.to_csv("my_sets.csv")